<a href="https://colab.research.google.com/github/pradhangaurv/Data-Scrapping/blob/master/musicplayermanager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
from collections import deque
import heapq

class Song:
    def __init__(self, title, artist, duration):
        self.title = title
        self.artist = artist
        self.duration = duration  # in minutes, float

    def __repr__(self):
        mins = int(self.duration)
        secs = int((self.duration - mins) * 60)
        return f"{self.title} by {self.artist} ({mins}:{secs:02d} min)"


class PlaylistNode:
    def __init__(self, song):
        self.song = song
        self.next = None
        self.prev = None


class Playlist:
    def __init__(self):
        self.head = None
        self.tail = None
        self.current = None

    def add_song(self, song):
        node = PlaylistNode(song)
        if not self.head:
            self.head = self.tail = node
            self.current = self.head
        else:
            self.tail.next = node
            node.prev = self.tail
            self.tail = node

    def next_song(self):
        if self.current and self.current.next:
            self.current = self.current.next
            return self.current.song
        return None

    def prev_song(self):
        if self.current and self.current.prev:
            self.current = self.current.prev
            return self.current.song
        return None

    def current_song(self):
        return self.current.song if self.current else None

    def start(self):
        self.current = self.head
        return self.current.song if self.current else None

    def show_playlist(self):
        print("\nPlaylist:")
        node = self.head
        idx = 1
        while node:
            marker = "<- current" if node == self.current else ""
            print(f"{idx}. {node.song} {marker}")
            node = node.next
            idx += 1


class MusicPlayer:
    def __init__(self):
        self.library = []
        self.playlist = Playlist()
        self.queue = []
        self.party_mode = []
        self.history = []
        self.repeat = False

    def add_to_library(self, title, artist, duration):
        self.library.append(Song(title, artist, duration))

    def show_library(self):
        print("\nLibrary:")
        for i, song in enumerate(self.library, 1):
            print(f"{i}. {song}")

    def add_to_playlist(self, index):
        if 0 <= index < len(self.library):
            self.playlist.add_song(self.library[index])

    def play_next(self):
        if self.queue:
            song = self.queue.pop(0)
        elif self.party_mode:
            song = self.party_mode.pop(0)
        else:
            song = self.playlist.next_song()

        if song:
            self.history.append(song)
            print(f"Now playing: {song}")
        elif self.repeat and self.playlist.head:
            self.playlist.current = self.playlist.head
            song = self.playlist.current.song
            self.history.append(song)
            print(f"Repeating playlist: {song}")
        else:
            print("No more songs to play.")

    def play_prev(self):
        song = self.playlist.prev_song()
        if song:
            self.history.append(song)
            print(f"Now playing: {song}")
        else:
            print("No previous song.")

    def replay_last(self):
        if self.history:
            song = self.history[-1]
            print(f"Replaying: {song}")
        else:
            print("No history yet.")

    def add_to_queue(self, index):
        if 0 <= index < len(self.library):
            self.queue.append(self.library[index])
            print(f"Queued {self.library[index]} for next play.")

    def add_to_party(self, index, votes):
        if 0 <= index < len(self.library):
            song = self.library[index]
            self.party_mode.append((votes, song))
            self.party_mode.sort(key=lambda x: -x[0])
            self.party_mode = [s for _, s in self.party_mode]
            print(f"Added {song} to party mode with {votes} votes.")

    def show_history(self):
        print("\nListening History:")
        if not self.history:
            print("No songs played yet.")
        for i, song in enumerate(reversed(self.history), 1):
            print(f"{i}. {song}")

    def toggle_repeat(self):
        self.repeat = not self.repeat
        print(f"Repeat mode {'ON' if self.repeat else 'OFF'}")


def build_player_with_initial_library():
    player = MusicPlayer()
    songs = [
        ("Zariya", "AR RAHMAN", 3.15),
        ("Fein", "Travis Scott, Playboi Carti", 7.12),
        ("Goosebumps", "Travis Scott", 4.11),
        ("Open Arms", "Travis Scott, SZA", 4.02),
        ("My Eyes", "Travis Scott", 4.13),
        ("Traitor", "Olivia Rodrigo", 3.50),
        ("Deja Vu", "Olivia Rodrigo", 3.36),
        ("Gods Plan", "Drake", 3.19),
    ]
    for title, artist, duration in songs:
        player.add_to_library(title, artist, duration)
        player.add_to_playlist(len(player.library) - 1)
    player.playlist.start()
    return player


def main():
    player = build_player_with_initial_library()

    menu_options = {
        "1": ("Show Library", lambda: player.show_library()),
        "2": ("Play Next", lambda: player.play_next()),
        "3": ("Play Previous", lambda: player.play_prev()),
        "4": ("Replay Last Song", lambda: player.replay_last()),
        "5": ("Add Song to Queue", lambda: add_to_queue_ui(player)),
        "6": ("Add Song to Party Mode", lambda: add_to_party_ui(player)),
        "7": ("Show History", lambda: player.show_history()),
        "8": ("Toggle Repeat Mode", lambda: player.toggle_repeat()),
        "9": ("Show Playlist", lambda: player.playlist.show_playlist()),
        "0": ("Exit", None)
    }

    while True:
        now_playing = player.playlist.current_song()
        print("\nNow playing:", now_playing if now_playing else "Nothing")
        print("\n--- Music Player Menu ---")
        for k, (desc, _) in menu_options.items():
            print(f"{k}. {desc}")

        choice = input("\nEnter choice: ").strip()
        if choice == "0":
            print("Exiting...")
            break
        elif choice in menu_options:
            action = menu_options[choice][1]
            if action:
                action()
        else:
            print("Invalid choice.")


def add_to_queue_ui(player):
    player.show_library()
    try:
        index = int(input("Enter song number to queue: ")) - 1
        player.add_to_queue(index)
    except:
        print("Invalid input.")


def add_to_party_ui(player):
    player.show_library()
    try:
        index = int(input("Enter song number for party mode: ")) - 1
        votes = int(input("Enter number of votes: "))
        player.add_to_party(index, votes)
    except:
        print("Invalid input.")


if __name__ == "__main__":
    main()



Now playing: Zariya by AR RAHMAN (3:08 min)

--- Music Player Menu ---
1. Show Library
2. Play Next
3. Play Previous
4. Replay Last Song
5. Add Song to Queue
6. Add Song to Party Mode
7. Show History
8. Toggle Repeat Mode
9. Show Playlist
0. Exit

Enter choice: 0
Exiting...
